<a href="https://colab.research.google.com/github/ammar0211/Mini-Projects/blob/master/Data_Analyst_Internship_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RESTAURANT LIST

INSTALLING LIBRARIES

In [1]:
pip install unicodecsv

  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp36-none-any.whl size=10767 sha256=28de67a16e7842d31bde81c1edf44d35fb7569098ee2058dec79a529c7ec6aa2
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
Successfully built unicodecsv


In [2]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 4.8MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubunt

IMPORTING LIBRARIES

In [0]:
import os,sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [0]:
#Libraries used for Data Extraction
import requests,re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import lxml
from lxml import html,etree
import time
from bs4 import BeautifulSoup

In [0]:
#Libraries used for Data Analysis
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from collections import Counter

In [6]:
#Setting Options for Chrome WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Importing the webdriver
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


ALLIED FUNCTIONS

In [0]:
#Function for Calculation of Polarity Score
def polarity_counter(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text.lower())
    filtered_tokens=[w for w in tokens if not w in stop_word]
    total_word_count=len(filtered_tokens)
    pos_score=replaceMultiple(" ".join(filtered_tokens),positive,"++POS++").count("++POS++")
    neg_score=replaceMultiple(" ".join(filtered_tokens),negative,"--NEG--").count("--NEG--")
    polarity_score=(pos_score-neg_score)/(pos_score+neg_score+0.000001)

    return polarity_score

In [0]:
#Extracter function for extracting webpages using BeautifulSoup
def extracter(URL):
    response=requests.get(URL).content
    return BeautifulSoup(response,'lxml')

In [0]:
#Cleaning Names/Titles of Restaurant
def clean_name(name):
    num=set(['0','1','2','3','4','5','6','7','8','9'])
    if list(name)[0] in num:
        a=list(name)
        while a[0]!=' ':
            a.pop(0)
        a.pop(0)
        return (''.join(a))
    return name

In [0]:
#Clean Review Paragraph Text
def clean_text(df,col):
  df.iloc[:,col]=df.iloc[:,col].apply(lambda x: x.lower().strip())
  df.iloc[:,col]=df.iloc[:,col].apply(lambda x: re.sub('[^a-zA-Z0-9]',' ',x))

In [0]:
# function to check if the button is on the page, to avoid miss-click problem
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [0]:
# Function for replacing multiple values at once in a text.\
def replaceMultiple(mainString, toBeReplaces, newString):
    for elem in toBeReplaces :
        if elem in mainString :
            mainString = mainString.replace(elem, newString)
    
    return  mainString

EXTRACTION FUNCTION

In [0]:
#Functin for Extracting all the restaurant List from the TripAdvisor website by using URL and providing Maximum Pages to look
def get_restaurant(url,max_page=None):
  restaurantList=[]
  driver.get(url)
  soup=BeautifulSoup(driver.page_source,'lxml')
  page_count=max_page if max_page else max(map(int,html.fromstring(str(soup)).xpath('//div[contains(@class,"deckTools")]//a[contains(@class,"pageNum taLnk")]//text()')))

  for i in range(0,page_count):
    soup=BeautifulSoup(driver.page_source,'lxml')
    restaurant_lists=html.fromstring(str(soup)).xpath('//div[contains(@class,"restaurants-list")]//div[contains(@data-test,"list_item")]')
    for restaurant in restaurant_lists:
      XPATH_NAME='.//div[contains(@class,"infoWrapper")]//a[contains(@class,"restaurantName")]//text()'
      XPATH_LINK='.//div[contains(@class,"infoWrapper")]//a[contains(@class,"restaurantName")]/@href'
      XPATH_REVIEWS='.//div[contains(@class,"infoWrapper")]//span[contains(@class,"userReviewCount")]//text()'

      raw_xpath_name=restaurant.xpath(XPATH_NAME)
      raw_xpath_link=restaurant.xpath(XPATH_LINK)
      raw_xpath_reviews=restaurant.xpath(XPATH_REVIEWS)

      prefix='https://www.tripadvisor.in'
      URL=prefix+raw_xpath_link[0] if raw_xpath_name else None
      name=clean_name(''.join(raw_xpath_name[0])) if raw_xpath_name else None
      reviewCount=raw_xpath_reviews[0].split(' ')[0] if raw_xpath_reviews else None

      data = {
          'Restaurant Name':name,
          'URL':URL,
          'REVIEWS':reviewCount
      }
      restaurantList.append(data)

    if (check_exists_by_xpath('*//div[contains(@class,"js_pageLinks")]//a[contains(@class,"nav next")]')):
          driver.find_element_by_xpath('//div[contains(@class,"js_pageLinks")]//a[contains(@class,"nav next")]').click()
          time.sleep(5)
  return restaurantList


In [0]:
#Functin for Extracting all the restaurant reviews from the TripAdvisor website by using URL and providing Maximum Pages to look
def get_review(url,max_page=None):
  reviewList=[]
  driver.get(url)
  soup=BeautifulSoup(driver.page_source,'lxml')
  page=max_page if max_page else max(map(int,html.fromstring(str(soup)).xpath('//div[contains(@class,"ui_pagination")]//a[contains(@class,"pageNum")]//text()')))
  for i in range(page):
      if (check_exists_by_xpath("//span[@class='taLnk ulBlueLinks']")):
        driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
      soup=BeautifulSoup(driver.page_source,'lxml')
      review_lists=html.fromstring(str(soup)).xpath('.//div[@id="REVIEWS"]//div[contains(@class,"review-container")]')
      for review in review_lists:
        XPATH_REVIEW_ID='.//div[contains(@class,"reviewSelector")]/@data-reviewid'
        XPATH_RATING='.//div[contains(@class,"reviewSelector")]//span[contains(@class,"ui_bubble_rating")]/@class'
        XPATH_REVIEW_TITLE='.//div[contains(@class,"reviewSelector")]//span[@class="noQuotes"]//text()'
        XPATH_REVIEW_DATE='.//div[contains(@class,"reviewSelector")]//div[@data-prwidget-name="reviews_stay_date_hsx"]/text()'
        XPATH_PARAGRAPH='.//div[contains(@class,"reviewSelector")]//div[@data-prwidget-name="reviews_text_summary_hsx"]//p[@class="partial_entry"]//text()'
        
        raw_review_id=review.xpath(XPATH_REVIEW_ID)
        raw_rating=review.xpath(XPATH_RATING)
        raw_review_title=review.xpath(XPATH_REVIEW_TITLE)
        raw_review_date=review.xpath(XPATH_REVIEW_DATE)
        raw_review_paragraph=review.xpath(XPATH_PARAGRAPH)
        
        ID=raw_review_id[0] if raw_review_id else None
        rating=list(raw_rating[0])[-2] if raw_rating else None
        title=raw_review_title[0] if raw_review_title else None
        date=raw_review_date[0] if raw_review_date else None
        paragraph=str(raw_review_paragraph[0]).replace('\n',' ') if raw_review_paragraph else None
        
        data = {
            'URL':url,
            'ID': ID,
            'Rating':rating,
            'Title':title,
            'Date':date,
            'Review':paragraph,
        }
        reviewList.append(data)

      if (check_exists_by_xpath('*//div[contains(@class,"ui_pagination")]//a[contains(@class,"nav next")]')):
        driver.find_element_by_xpath('//div[contains(@class,"ui_pagination")]//a[contains(@class,"nav next")]').click()
        time.sleep(5)
  return reviewList

In [0]:
#Defining stopwords,positive words and negative words for sentiment analysis dowloaded from Cambridge University Press
stop_word=list(pd.read_csv('https://raw.githubusercontent.com/ammar0211/sentiment_data/master/StopWords_GenericLong.txt',delimiter=' ',header=None)[0])
positive=list(pd.read_csv('https://raw.githubusercontent.com/ammar0211/sentiment_data/master/positive-words.txt',delimiter=' ',header=None)[0])
negative=list(pd.read_csv('https://raw.githubusercontent.com/ammar0211/sentiment_data/master/negative-words.txt',delimiter=' ',header=None)[0])

In [0]:
RESTAURANT_LIST=pd.DataFrame(get_restaurant('https://www.tripadvisor.in/Restaurants-g304551-New_Delhi_National_Capital_Territory_of_Delhi.html',2))

In [17]:
RESTAURANT_LIST

,Restaurant Name,URL,REVIEWS
0,Detroit Garage Bar & Kitchen,https://www.tripadvisor.in/Restaurant_Review-g...,39
1,Spring,https://www.tripadvisor.in/Restaurant_Review-g...,576
2,Thyme,https://www.tripadvisor.in/Restaurant_Review-g...,697
3,Dilli 32,https://www.tripadvisor.in/Restaurant_Review-g...,243
4,The Hong Kong Club,https://www.tripadvisor.in/Restaurant_Review-g...,"1,035"
...,...,...,...
57,Akira Back,https://www.tripadvisor.in/Restaurant_Review-g...,332
58,Spezia Bistro,https://www.tripadvisor.in/Restaurant_Review-g...,75
59,Bukhara,https://www.tripadvisor.in/Restaurant_Review-g...,"4,191"
60,Kebabs & Kurries,https://www.tripadvisor.in/Restaurant_Review-g...,126


In [0]:
#Creating a list of dictionaries containing all the reviews of all the restaurant from the RESTAURANT_LIST dataframe
reviewList=[]
for url in RESTAURANT_LIST.URL:
  reviewList+=get_review(url,2)

#Converting the review list into DataFrame
DF=pd.DataFrame(reviewList)

In [0]:
#Cleaning the review paragraph
clean_text(DF,5)

In [20]:
DF.head()

,URL,ID,Rating,Title,Date,Review
0,https://www.tripadvisor.in/Restaurant_Review-g...,724691556,5,"Detroit, Place Worth Visiting For All The Food...",November 2019,the classy vintage vibes with those amazing c...
1,https://www.tripadvisor.in/Restaurant_Review-g...,724212126,4,"Great ambience, good food",October 2019,a theme based interior with all car related ...
2,https://www.tripadvisor.in/Restaurant_Review-g...,717122952,5,AMAZING PLACE!,October 2019,newly opened place in cp and i believe this pl...
3,https://www.tripadvisor.in/Restaurant_Review-g...,715745604,4,Vintage car,October 2019,detroit garage bar kitchen is an amazing pla...
4,https://www.tripadvisor.in/Restaurant_Review-g...,714965148,4,"Nice Ambience, But Average Food taste!",September 2019,nice place with great decor and music howeve...


In [0]:
#Calculating the polarity score
DF['Polarity Score']=DF.Review.apply(lambda x: polarity_counter(x))

Hence, Most postive would be ReviewID 720976868 ('errrrmmm not sure how to fully express my annoyance and anger after learning that we were charged wrong   way too much') with -1 Polarity Score 
Most positive would be ReviewID 705010600 ('thank you team bling    visited first time and after exploring whole of aerocity your team was the best   being a member so many other hotel chains   you proved why pullman is best and deserving   all thanks to mr  gaurav for customizing our drinks as per taste and make our visit a memorable one      would definitely come again   thanks again') with +1 Polarity among many others with same score.


In [22]:
#Calculating the Most Frequent Words
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize((" ".join(DF.Review)).lower())
filtered_tokens=[w for w in tokens if not w in stop_word]
word_count=Counter(filtered_tokens)
word_count.most_common(2)

[('food', 766), ('service', 498)]

Therefore, food is the most common word with 785 occurances throughout the data followed by services with 521 occurances.

In [0]:
DF.drop(columns=['ID'],axis=1,inplace=True)

In [0]:
#Converting th review file into CSV
DF.to_csv('Data Analyst Internship Submission.csv',header=True,index=False)